In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import layers
from keras.layers import Dense, Input, Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout,GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as k

Using TensorFlow backend.


In [2]:
#Let's create CNN Model
def keras_model(img_x_shape,img_y_shape):
    num_of_classes = 12
    model = Sequential()
    model.add(Conv2D(32,(5,5), input_shape=(img_x_shape,img_y_shape,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64, (5,5),activation='sigmoid'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath = "emoji.h5"
    chckpnt = ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list = [chckpnt]
    
    return model, callbacks_list

In [3]:
df = pd.read_csv("train_foo.csv")
df = np.array(df)
np.random.shuffle(df)
x = df[:,1:]
y = df[:,0]
index_part = int(0.8*x.shape[0])

In [4]:
#Data Preprocesing
y = y.reshape(y.shape[0],1) #Reshaping y for y_train & y_test
x_train = x[0:index_part,:]
x_train = x_train / 255.
y_train = y[0:index_part:,:]
y_train = y_train.T

x_test = x[index_part:,:]
x_test = x_test / 255.
y_test = y[index_part:,:]
y_test = y_test.T

In [5]:
#Define Image pixel shape
img_x_shape = 50
img_y_shape = 50

In [6]:
#Make categorical output for Softmax
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [7]:
#Reshape the y_train and y_test vectors
y_train = y_train.reshape(y_train.shape[1],y_train.shape[2])
y_test = y_test.reshape(y_test.shape[1],y_test.shape[2])

In [8]:
#Reshape the x_train and x_test as 50X50 pixel Image 
x_train = x_train.reshape(x_train.shape[0],50,50,1)
x_test = x_test.reshape(x_test.shape[0],50,50,1)

In [9]:
#Let's train
model, callbacks_list = keras_model(img_x_shape,img_y_shape)
model.fit(x_train,y_train, validation_data=(x_test,y_test),epochs=10,batch_size=64,callbacks=callbacks_list)
scores = model.evaluate(x_test,y_test,verbose=0)
print("CNN Error: %.2f%%" %(100-scores[1]*100))

model.save('emoji.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 10559 samples, validate on 2640 samples
Epoch 1/10
10559/10559 [==============================] - 12s 1ms/step - loss: 0.5879 - acc: 0.8276 - val_loss: 0.0119 - val_acc: 0.9966

Epoch 00001: val_acc improved from -inf to 0.99659, saving model to emoji.h5
Epoch 2/10
10559/10559 [==============================] - 12s 1ms/step - loss: 0.0158 - acc: 0.9962 - val_loss: 0.0018 - val_acc: 1.0000

Epoch 00002: val_acc improved from 0.99659 to 1.00000, saving model to emoji.h5
Epoch 3/10
10559/10559 [==============================] - 11s 1ms/step - loss: 0.0047 - acc: 0.9991 - val_loss: 3.3506e-04 - val_acc: 1.0000

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/10
10559/10559 [==============================] - 11s 1ms/step - loss: 0.0018 - acc